**1. 데이터 불러오기**

In [1]:
# 데이터 불러오기
import pandas as pd
original_data = pd.read_csv('./total_price.csv')

# 불필요한 열 제거
del original_data['Unnamed: 0']

# 날짜를 인덱스로 변경
original_data = original_data.set_index('date')

# l1 ~ l4 Line Setting
original_data['l1'] = original_data['l1'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l2'] = original_data['l2'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l3'] = original_data['l3'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l4'] = original_data['l4'].apply(lambda x : str(x)).apply(lambda x : x[1:])

# Data Head
original_data

,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,code
date,,,,,,,,,,,,,,,
20160622,10150,10150,9780,9830,315346,2.586,0.538,2.477,10175.0,9990.0,9805.0,9620.0,555.0,6,20
20160623,9710,9870,9510,9730,293348,2.778,1.429,47.114,9980.0,9800.0,9620.0,9440.0,540.0,6,20
20160624,9840,9910,8700,9080,621895,4.162,0.960,27.960,10100.0,9495.0,8890.0,8285.0,1815.0,20,20
20160627,8750,9480,8750,9400,334886,3.940,0.880,23.643,9805.0,9440.0,9075.0,8710.0,1095.0,12,20
20160628,9210,9770,9210,9760,282254,3.940,0.880,23.643,10045.0,9765.0,9485.0,9205.0,840.0,9,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200219,14750,14880,14750,14850,1090,2857.000,10.000,79.010,14930.0,14865.0,14800.0,14735.0,195.0,1,241180
20200220,14850,15005,14790,14850,3900,2732.000,20.000,79.751,15035.0,14927.5,14820.0,14712.5,322.5,2,241180
20200221,14695,14845,14695,14845,1706,2722.000,20.000,79.751,14920.0,14845.0,14770.0,14695.0,225.0,2,241180


**2. 데이터 전처리**

In [2]:
# 라이브러리
import numpy as np
import pandas as pd

# 종목 리스트
codes = original_data.code.unique()

# 최종 데이터프레임을 저장할 공간
dataset = pd.DataFrame()

# 중간 결과 집계
COUNT = 0

# 종목별 전처리 진행
for code in codes : 
    
    # 날짜별로 데이터 재정렬
    data = original_data[original_data['code'] == code].reset_index().sort_values(by=['date'])
    
    # open, high, low, close 를 모두 실수화
    data['open'] = data['open'].apply(lambda x : np.float(x))
    data['high'] = data['high'].apply(lambda x : np.float(x))
    data['low'] = data['low'].apply(lambda x : np.float(x))
    data['close'] = data['close'].apply(lambda x : np.float(x))

    # Zone 지정
    data['zone']=100*(data['close']-data['close'].shift(-1))/data['close'].shift(-1)
    
    def zone(x) : 
        if x < -10 : return "-30 ~ -10"
        elif x < -5 : return "-10 ~ -3"
        elif x < -1 : return "-3 ~ -1"
        elif x < +1 : return "-1 ~ +1"
        elif x < +5 : return "+1 ~ +3"
        elif x < +10 : return "+3 ~ +10"
        elif x < +20 : return "+10 ~ +20"
        else : return "+10 ~ +30"
            
    data['zone'] = data['zone'].apply(lambda x : zone(x))
    
    # open과 high와의 차이를 open으로 나누고 반올림
    data['open_high'] = (data['high'] - data['open']) / data['open']
    data['open_high'] = 100 * data['open_high'].apply(lambda x : round(x,2))

    # open과 low와의 차이를 open으로 나누고 반올림
    data['open_low'] = (data['low'] - data['open']) / data['open']
    data['open_low'] = 100 * data['open_low'].apply(lambda x : round(x,2))
    
    # open과 close의 차이를 open으로 나누고 반올림
    data['open_close'] = (data['close'] - data['open']) / data['open']
    data['open_close'] = 100 * data['open_close'].apply(lambda x : round(x,2))
    
    # open 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['open'] = 100 * (data['open']-data['open'].shift(1)) / data['open'].shift(1)
    data['open'] = data['open'].apply(lambda x : round(x,2))
    
    # high 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['high'] = 100*(data['high']-data['high'].shift(1)) / data['high'].shift(1)
    data['high'] = data['high'].apply(lambda x : round(x,2))

    # low 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['low'] = 100*(data['low']-data['low'].shift(1))/data['low'].shift(1)
    data['low'] = data['low'].apply(lambda x : round(x,2))

    # close 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['close'] = 100*(data['close']-data['close'].shift(1))/data['close'].shift(1)
    data['close'] = data['close'].apply(lambda x : round(x,2))

    # trading_volume 실수화한 다음, 반올림 (로그변환 하기 전에 +1 해줌)
    data['volume'] = data['trading_volume'].apply(lambda x : np.float(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x + 1))
    data['volume'] = data['volume'].apply(lambda x : round(x,2))
    

    # Score는 실수화하고, 로그변환만 진행하고 반올림 (로그변환 하기 전에 +1 해줌)
    data['score'] = data['score'].apply(lambda x : np.float(x))
    data['score'] = data['score'].apply(lambda x : np.log(x +1))
    data['score'] = data['score'].apply(lambda x : round(x,2))

    # index는 실수화하고, 로그변환만 진행하고 반올림 (로그변환 하기 전에 +1 해줌)
    data['index'] = data['index'].apply(lambda x : np.float(x))
    data['index'] = data['index'].apply(lambda x : np.log(x + 1))
    data['index'] = data['index'].apply(lambda x : round(x,2))
    
    # probability는 그대로 이용하고 반올림
    data['probability'] = data['probability'].apply(lambda x : np.float(x))
    data['probability'] = data['probability'].apply(lambda x : round(x,2))

    # l1 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l1'] = data['l1'].apply(lambda x : np.float(x))
    data['l1'] = 100*(data['l1']-data['l1'].shift(1))/data['l1'].shift(1)
    data['l1'] = data['l1'].apply(lambda x : np.float(x))
    data['l1'] = data['l1'].apply(lambda x : round(x,2))
    
    # l2 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l2'] = data['l2'].apply(lambda x : np.float(x))
    data['l2'] = 100*(data['l2']-data['l2'].shift(1))/data['l2'].shift(1)
    data['l2'] = data['l2'].apply(lambda x : np.float(x))
    data['l2'] = data['l2'].apply(lambda x : round(x,2))

    # l3 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l3'] = data['l3'].apply(lambda x : np.float(x))
    data['l3'] = 100*(data['l3']-data['l3'].shift(1))/data['l3'].shift(1)
    data['l3'] = data['l3'].apply(lambda x : np.float(x))
    data['l3'] = data['l3'].apply(lambda x : round(x,2))

    # l1 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l4'] = data['l4'].apply(lambda x : np.float(x))
    data['l4'] = 100*(data['l4']-data['l4'].shift(1))/data['l4'].shift(1)
    data['l4'] = data['l4'].apply(lambda x : np.float(x))
    data['l4'] = data['l4'].apply(lambda x : round(x,2))

    # lgap는 로그변환만하고, 반올림 (로그변환 하기 전에 +1 해줌)
    data['lgap'] = data['lgap'].apply(lambda x : np.float(x))
    data['lgap'] = data['lgap'].apply(lambda x : np.log(x + 1))
    data['lgap'] = data['lgap'].apply(lambda x : round(x,2))

    # lrate는 그대로 이용
    data['lrate']=data['lrate'].apply(lambda x : np.float(x))

    # 모델에 불필요한 변수 제거
    del data['date']
    del data['code']
    
    # 전처리 과정에서 생긴 결측값 제거
    data = data.dropna()
    
    # 최종 데이터셋에 추가
    dataset = dataset.append(data)
    
    # data 리셋
    data = pd.DataFrame()
    
    # 중간결과 출력
    COUNT += 1
    if COUNT % 100 == 0 : 
        print('{}번째 종목 전처리 진행중.....' .format(COUNT))
        
# 이상값 제거 (전일대비 OHLC가 35% 이상 뛴 종목)
dataset = dataset[dataset['open'] < 35.00]
dataset = dataset[dataset['high'] < 35.00]
dataset = dataset[dataset['low'] < 35.00]
dataset = dataset[dataset['close'] < 35.00]

# # 데이터프레임 순서 변경
dataset = dataset[['open', 'high', 'low', 'close', 'volume', 'score', 'index', 'probability',\
                   'l1', 'l2', 'l3', 'l4', 'lgap', 'lrate', 'open_high', 'open_low', 'open_close', 'zone']]

100번째 종목 전처리 진행중.....
200번째 종목 전처리 진행중.....
300번째 종목 전처리 진행중.....
400번째 종목 전처리 진행중.....
500번째 종목 전처리 진행중.....
600번째 종목 전처리 진행중.....
700번째 종목 전처리 진행중.....
800번째 종목 전처리 진행중.....
900번째 종목 전처리 진행중.....
1000번째 종목 전처리 진행중.....
1100번째 종목 전처리 진행중.....
1200번째 종목 전처리 진행중.....
1300번째 종목 전처리 진행중.....
1400번째 종목 전처리 진행중.....
1500번째 종목 전처리 진행중.....
1600번째 종목 전처리 진행중.....
1700번째 종목 전처리 진행중.....
1800번째 종목 전처리 진행중.....
1900번째 종목 전처리 진행중.....
2000번째 종목 전처리 진행중.....
2100번째 종목 전처리 진행중.....


**3. 데이터 전처리 결과**

In [3]:
dataset

,open,high,low,close,volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,open_high,open_low,open_close,zone
1,-4.33,-2.76,-2.76,-1.02,12.59,1.33,0.89,47.11,-1.92,-1.90,-1.89,-1.87,6.29,6.0,2.0,-2.0,0.0,+3 ~ +10
2,1.34,0.41,-8.52,-6.68,13.34,1.64,0.67,27.96,1.20,-3.11,-7.59,-12.24,7.50,20.0,1.0,-12.0,-8.0,-3 ~ -1
3,-11.08,-4.34,0.57,3.52,12.72,1.60,0.63,23.64,-2.92,-0.58,2.08,5.13,7.00,12.0,8.0,0.0,7.0,-3 ~ -1
4,5.26,3.06,5.26,3.83,12.55,1.60,0.63,23.64,2.45,3.44,4.52,5.68,6.73,9.0,6.0,0.0,6.0,-1 ~ +1
5,6.95,3.38,5.32,-0.10,12.77,1.60,0.63,23.64,0.80,1.64,2.53,3.48,6.40,6.0,3.0,-2.0,-1.0,-3 ~ -1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,-0.37,-0.47,-0.17,0.34,6.99,7.96,2.40,79.01,-0.22,-0.07,0.08,0.24,5.28,1.0,1.0,0.0,1.0,-1 ~ +1
898,0.68,0.84,0.27,0.00,8.27,7.91,3.04,79.75,0.70,0.42,0.14,-0.15,5.78,2.0,1.0,-0.0,0.0,-1 ~ +1
899,-1.04,-1.07,-0.64,-0.03,7.44,7.91,3.04,79.75,-0.76,-0.55,-0.34,-0.12,5.42,2.0,1.0,0.0,1.0,+1 ~ +3
900,-0.31,-1.11,-0.92,-1.48,6.19,7.90,1.95,77.30,-1.39,-1.30,-1.20,-1.11,5.20,1.0,0.0,-1.0,-0.0,-3 ~ -1


In [4]:
dataset['zone'].value_counts()

-1 ~ +1      917587
+1 ~ +3      464532
-3 ~ -1      400994
+3 ~ +10      49751
-10 ~ -3      45136
-30 ~ -10     12382
+10 ~ +20      7819
+10 ~ +30      3598
Name: zone, dtype: int64

In [18]:
A = 100 *dataset['zone'].value_counts()[0] / 1901799
B = 100 *dataset['zone'].value_counts()[1] / 1901799
C = 100 *dataset['zone'].value_counts()[2] / 1901799
D = 100 *dataset['zone'].value_counts()[3] / 1901799
E = 100 *dataset['zone'].value_counts()[4] / 1901799
F = 100 *dataset['zone'].value_counts()[5] / 1901799
G = 100 *dataset['zone'].value_counts()[6] / 1901799
H = 100 *dataset['zone'].value_counts()[7] / 1901799

print('-1 ~ +1 비율 = {:.2f}%' .format(A))
print('+1 ~ +3 비율 = {:.2f}%' .format(B))
print('-3 ~ -1 비율 = {:.2f}%' .format(C))
print('+3 ~ +10 비율 = {:.2f}%' .format(D))
print('-10 ~ -3 비율 = {:.2f}%' .format(E))
print('-30 ~ -10  비율 = {:.2f}%' .format(F))
print('+10 ~ +20 비율 = {:.2f}%' .format(G))
print('+10 ~ +30 비율 = {:.2f}%' .format(H))

-1 ~ +1 비율 = 48.25%
+1 ~ +3 비율 = 24.43%
-3 ~ -1 비율 = 21.08%
+3 ~ +10 비율 = 2.62%
-10 ~ -3 비율 = 2.37%
-30 ~ -10  비율 = 0.65%
+10 ~ +20 비율 = 0.41%
+10 ~ +30 비율 = 0.19%


In [6]:
round(dataset.describe(), 2)

,open,high,low,close,volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,open_high,open_low,open_close
count,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00,1901799.00
mean,-0.00,0.01,-0.01,0.01,10.79,3.28,1.47,52.50,0.03,0.00,-0.01,0.01,5.75,5.55,1.87,-1.81,-0.06
std,3.16,3.34,2.64,2.95,2.79,3.18,1.08,26.68,3.83,2.86,2.45,3.11,1.61,4.88,2.90,2.02,2.83
min,-94.54,-93.68,-95.92,-95.92,0.00,0.00,0.01,0.00,-93.68,-94.80,-95.92,-97.04,0.00,0.00,0.00,-46.00,-45.00
25%,-1.34,-1.16,-0.98,-1.16,9.69,0.85,0.69,30.00,-1.34,-1.10,-0.92,-1.05,4.86,3.00,0.00,-3.00,-1.00
50%,0.00,-0.11,0.00,0.00,11.19,1.80,1.05,57.52,-0.25,-0.16,0.00,0.10,5.71,4.00,1.00,-1.00,0.00
75%,1.17,0.79,0.94,0.96,12.47,6.40,2.20,78.46,0.77,0.71,0.93,1.18,6.72,7.00,2.00,0.00,1.00
max,34.99,32.48,34.89,32.23,20.55,15.15,4.62,79.99,52.79,31.00,32.40,76.88,12.62,128.00,81.00,0.00,67.00


In [7]:
pd.pivot_table(dataset, index=['zone'], values=['open','high','low','close'], aggfunc='mean')

,close,high,low,open
zone,,,,
+1 ~ +3,0.068613,-0.006566,-0.014092,0.015660
+10 ~ +20,1.017364,1.637313,-0.106372,0.772834
+10 ~ +30,-0.387957,-0.096598,-1.704411,-1.419113
+3 ~ +10,-0.013286,0.198753,-0.216545,0.099076
-1 ~ +1,0.004154,-0.018521,0.024136,0.009856
-10 ~ -3,0.220830,0.381326,0.066815,0.094208
-3 ~ -1,-0.146605,-0.044961,-0.076698,-0.095749
-30 ~ -10,1.836933,1.934098,0.513870,0.565833


In [8]:
pd.pivot_table(dataset, index=['zone'], values=['volume','score','index','probability'], aggfunc='mean')

,index,probability,score,volume
zone,,,,
+1 ~ +3,1.465210,52.320333,3.213184,11.398402
+10 ~ +20,1.502748,53.038529,3.040547,13.154364
+10 ~ +30,1.469647,51.724233,3.066306,11.883708
+3 ~ +10,1.493856,52.808631,3.072970,12.616808
-1 ~ +1,1.463451,52.290332,3.345987,9.993443
-10 ~ -3,1.493377,52.893166,3.091008,12.331521
-3 ~ -1,1.506728,53.128024,3.297997,11.364527
-30 ~ -10,1.414520,51.168690,2.811996,12.858134


In [9]:
pd.pivot_table(dataset, index=['zone'], values=['l1','l2','l3','l4'], aggfunc='mean')

,l1,l2,l3,l4
zone,,,,
+1 ~ +3,0.046540,0.020191,0.018484,0.050246
+10 ~ +20,2.202967,1.291361,0.420040,-0.363764
+10 ~ +30,0.552393,-0.258885,-1.062913,-1.830536
+3 ~ +10,0.326821,0.068653,-0.138915,-0.267849
-1 ~ +1,-0.021051,-0.012864,0.009159,0.049037
-10 ~ -3,0.477037,0.281189,0.125960,0.032681
-3 ~ -1,-0.066847,-0.105540,-0.121226,-0.105694
-30 ~ -10,2.582882,1.849077,1.153356,0.532854


In [10]:
pd.pivot_table(dataset, index=['zone'], values=['lgap','lrate'], aggfunc='mean')

,lgap,lrate
zone,,
+1 ~ +3,5.989003,6.193879
+10 ~ +20,6.428456,14.635120
+10 ~ +30,6.037690,11.509728
+3 ~ +10,6.244502,10.310667
-1 ~ +1,5.460388,4.291023
-10 ~ -3,6.199187,9.247740
-3 ~ -1,5.975238,6.240864
-30 ~ -10,6.097146,12.921176


In [11]:
pd.pivot_table(dataset, index=['zone'], values=['open_high','open_low','open_close'], aggfunc='mean')

,open_close,open_high,open_low
zone,,,
+1 ~ +3,-0.020821,2.082453,-2.029875
+10 ~ +20,0.560813,5.550326,-4.386111
+10 ~ +30,1.016120,4.481934,-3.147304
+3 ~ +10,-0.143977,3.634721,-3.265482
-1 ~ +1,-0.054381,1.388399,-1.436983
-10 ~ -3,0.086561,3.418070,-2.749335
-3 ~ -1,-0.191025,2.126009,-2.004761
-30 ~ -10,1.443709,5.606929,-3.245114


**4. 전처리된 데이터 저장**

In [12]:
dataset.to_csv('./final_dataset.csv')